# Section 1: Course Introduction

## 1.1 Course Structure

## 1.2 Technologies
Tech Stack : git, python django, django rest framework virtualbox, modheader(chrome extension), vagrant, atom

### 1. Development Server  
Vagrant helps us create a virtual development machine.  
Running code in a virtual dev machine has many benefits:  
  * easy to share the server with others  
  * exact same version of all requirements  
  * run exactly the same software as a real production server  
  * easily create and destroy server as needed  
  
### 2. Application code  
Layer #1 Programing Language - Python
   * write the logic for our application

Layer #2 Django Framework
   * web framework for Python
   * pre-defined set of code to perform common actions
   * stardardises applications
 
Layer #3 Django REST framework
   * provides set of features for making standard REST API's


### 3. Vagrant vs Docker

They both use virtualization technology to isolate application from local machine  
  
Docker has a steeper learning curve than Vagrant, has streamlined workflow, and requires supported OS.  
Used for Production. A docker file creates a docker image that can be ran on local or server machines.  
  
Vagrant allows you to manage VDE's. It uses a hypervisor (ie. Virtual Vox) to create a server.  
Contains streamlined but complete version of Linux OS. Not used for production. 

### 4. How to get the most out of this course
### 5. How to get help

# Section 2: Setting up your project

### 6. Windows: Installing technologies
### 7. macOS: Installing technologies
cheatsheet:  
https://github.com/LondonAppDev/build-a-backend-api-python-drf-beginner-cheat-sheet/blob/master/README.md

# Section 3: Creating a workspace

### 8. Creating a workspace

### 9. Creating a Git Project


In [ ]:
#Git Commands
#Initialize git in a folder  
git init  
git add .  
git commit -am "Commit Message"  

#create:  
README.md 

#.gitignore  
https://gist.github.com/LondonAppDev/dd166e24f69db4404102161df02a63ff

#LICENSE   
https://choosealicense.com/licenses/mit/

### 10. Creating a Git Project

In [ ]:
# Go to ssh folder
ls ~/.ssh

# Create a public/private rsa key with 4096 bytes
ssh-keygen -t rsa -b 4096 -C "huangjohn92@gmail.com"

# Go to github.com --> Settings --> SSH and GPG Keys
cat ~/.ssh/id_rsa.pub

# Copy and Paste (ssh till .com) into SSH keys/add new

# Create new repository on github
git remote add origin git@github.com:...
git push origin master

# Section 4: Creating a developement server

### 11. Creating a Vagrant File

In [ ]:
#create new vagrant image on terminal inside working folder
vagrant init ubuntu/bionic64

### 12. Configuring our Vagrant box    

In [ ]:
# replace Vagrantfile contents
https://gist.github.com/LondonAppDev/199eef145a21587ea866b69d40d28682

# -*- mode: ruby -*-
# vi: set ft=ruby :

# All Vagrant configuration is done below. The "2" in Vagrant.configure
# configures the configuration version (we support older styles for
# backwards compatibility). Please don't change it unless you know what
# you're doing.
Vagrant.configure("2") do |config|
 # The most common configuration options are documented and commented below.
 # For a complete reference, please see the online documentation at
 # https://docs.vagrantup.com.

 # Every Vagrant development environment requires a box. You can search for
 # boxes at https://vagrantcloud.com/search.
 config.vm.box = "ubuntu/bionic64"
 config.vm.box_version = "~> 20191107.0.0"

 config.vm.network "forwarded_port", guest: 8000, host: 8000

 config.vm.provision "shell", inline: <<-SHELL
   systemctl disable apt-daily.service
   systemctl disable apt-daily.timer
 
   sudo apt-get update
   sudo apt-get install -y python3-venv zip
   touch /home/vagrant/.bash_aliases
   if ! grep -q PYTHON_ALIAS_ADDED /home/vagrant/.bash_aliases; then
     echo "# PYTHON_ALIAS_ADDED" >> /home/vagrant/.bash_aliases
     echo "alias python='python3'" >> /home/vagrant/.bash_aliases
   fi
 SHELL
end

### 13. Running and connecting to our dev server

In [ ]:
# Load vagrant file and use Virtual box to create a virtual machine
vagrant up

# Connect to vritual machine
vagrant ssh

### 14. Running a Hello World script

In [ ]:
# go to vagrant directory which is synced to our working folder (wherever vagrantfile is)
cd /vagrant

touch hello_world.py
print('Hello World')
python hello_world.py

# Section 5: Creating a Django App


### 15. Create Python Virtual Environment

In [ ]:
python -m venv ~/env
source 

### 16. Create Python Virtual Environment

In [ ]:
#requirements.txt
django==2.2
djangorestframework==3.9.2

#activate virtual env
source ~/env/bin/activate

#to deactivate
deactivate

#install requirements (-r stands for requirements)
pip install -r requirements.txt

#pypi python package index
https://pypi.org/

### 17. Create a new Django project and app

In [ ]:
#create a new django project
# the . creates the project in the root of the working folder instead of creaing a new folder
django-admin.py startproject profiles_project .

#create new app
python manage.py startapp profiles_api

### 18. Enable our app in the Django settings file

In [ ]:
'''
in profiles_project
settings.py
'''

INSTALLED_APPS = [...
                 'rest_framework',
                 'rest_framework.authtoken',
                 'profiles_api',]

### 19. Test and commit our changes

In [ ]:
#to start server
python manage.py runserver 0.0.0.0:8000
    
#commit changes
git status
git add .
git commit -m "Created django project and app"
git push origin

# Section 6: Setup the Database


### 20. What are Django Models?

models describe the data we need for our app  
django uses models to set up and configure database  
each model maps to a specific table in our database  
django handles the relationship for us, so we dont need to interact with the db directly  

### 21. Create our user database model

In [ ]:
#django comes with default user model thats used for the standard authentication system and admin
https://docs.djangoproject.com/en/1.11/ref/models/fields/
    
#creating custom user model
https://docs.djangoproject.com/en/1.11/topics/auth/customizing/#auth-custom-user
    
#models.py
from django.db import models
from django.contrib.auth.models import AbstractBaseUser
from django.contrib.auth.models import PermissionsMixin


class UserProfile(AbstractBaseUser, PermissionsMixin):
    """Database model for users in the system"""
    email = models.EmailField(max_length=255, unique=True)
    name = models.CharField(max_length=255)
    is_active = models.BooleanField(default=True)
    is_staff = models.BooleanField(default=False)

    objects = UserProfileManager()

    USERNAME_FIELD = 'email'
    REQUIRED_FIELDS = ['name']

    def get_full_name(self):
        """Retrieve full name for user"""
        return self.name

    def get_short_name(self):
        """Retrieve short name of user"""
        return self.name

    def __str__(self):
        """Return string representation of user"""
        return self.email

### 22. Add a user model manager

In [ ]:
#add to models.py
from django.contrib.auth.models import BaseUserManager
...


class UserProfileManager(BaseUserManager):
    """Manager for user profiles"""

    def create_user(self, email, name, password=None):
        """Create a new user profile"""
        if not email:
            raise ValueError('Users must have an email address')

        email = self.normalize_email(email)
        user = self.model(email=email, name=name,)

        user.set_password(password)
        user.save(using=self._db)

        return user

    def create_superuser(self, email, name, password):
        """Create and save a new superuser with given details"""
        user = self.create_user(email, name, password)

        user.is_superuser = True
        user.is_staff = True
        user.save(using=self._db)

        return user

### 23. Set our custom user model

In [37]:
#configure project to use this model
#settings.py
AUTH_USER_MODEL = 'profiles_api.UserProfile'

### 24. Create migrations and sync DB

In [ ]:
#create django migration file
#django uses a migration file that stores all the steps required to make our database match our models
#every new/change model requires new migration file

python manage.py makemigrations profiles_api
python manage.py migrate

#push changes to github

# Section 7: Setup Django Admin

### 25. Creating a superuser

In [ ]:
#create super user
python manage.py createsuperuser

### 26. Enable Django Admin

In [ ]:
#admin.py
from django.contrib import admin
from profiles_api import models

admin.site.register(models.UserProfile)

### 27. Test Django Admin

In [ ]:
#in web browser
127.0.0.1:8000/admin

#push changes to github

# Section 8: Introduction to API Views

### 28. What is an APIView

Django REST Framework Views
https://www.django-rest-framework.org/api-guide/views/
1. APIView  
    most basic type of view  
    enables us to describe logic to make API endpoint  
    
    Uses standard HTTP Methods for functions  
    GET -return one ore more items  
    POST -create an item  
    PUT -update an item  
    PATCH -partially update an item  
    DELETE -delete an item  
    
    Perfect for implementing complex logic
    Calling other APIs
    Working with local files
2. ViewSet

### 29. Create first APIView

In [ ]:
#views.py
from rest_framework.views import APIView
from rest_framework.response import Response


class HelloApiView(APIView):
    """Test API View"""

    def get(self, request, format=None):
        """Returns a list of APIView features"""

        an_apiview = [
            'Uses HTTP methods as functions (get, post, patch, put, delete)',
            'Is similar to a traditional Django View',
            'Gives you the most control over your logic',
            'Is mapped manually to URLs',
        ]

        return Response({'message': 'Hello!', 'an_apiview': an_apiview})

### 30. Configure view URL

In [ ]:
#profiles-project --> urls.py
from django.urls import path, include

...

urlpatterns = [
    path('admin/', admin.site.urls),
    path('api/', include('profiles_api.urls'))
]

#profiles-api --> urls.py
from django.urls import path

from profiles_api import views


urlpatterns = [
    path('hello-view/', views.HelloApiView.as_view()),
]


URL Dispatcher : https://docs.djangoproject.com/en/1.11/topics/http/urls/  
Path function official docs: https://docs.djangoproject.com/en/2.2/ref/urls/#django.urls.path  
Include urls docs:  https://docs.djangoproject.com/en/2.2/ref/urls/#django.urls.include  

### 31. Testing out API View

In [ ]:
127.0.0.1:8000/api/hello_view/

#commite changes to github

### 32. Create a Serializer

In [ ]:
#A serializer is a feature that from the django REST framework that allows us to 
https://www.django-rest-framework.org/api-guide/serializers/
#change data inputs into python objects

#profiles_api --> serializers.py
from rest_framework import serializers


class HelloSerializer(serializers.Serializer):
    """Serializes a name field for testing out APIView"""
    name = serializers.CharField(max_length=10)

### 33. Add POST method to APIView

In [ ]:
#Add to views.py

from profiles_api import serializers

def post(self, request):
        """Create a hello message with our name"""
        serializer = self.serializer_class(data=request.data)

        if serializer.is_valid():
            name = serializer.validated_data.get('name')
            message = f'Hello {name}!'
            return Response({'message': message})
        else:
            return Response(
                serializer.errors,
                status=status.HTTP_400_BAD_REQUEST

#status-codes
https://www.django-rest-framework.org/api-guide/status-codes/

### 34. Test POST Function

In [ ]:
127.0.0.1:8000/api/hello_view/

### 35. Add PUT, PATCH and DELETE methods

In [ ]:
# Add to views.py

def put(self, request, pk=None):
    """Handle updating an object"""

    return Response({'method': 'PUT'})

def patch(self, request, pk=None):
    """Handle partial update of object"""

    return Response({'method': 'PATCH'})

def delete(self, request, pk=None):
    """Delete an object"""

    return Response({'method': 'DELETE'})

### 36. Test the PUT, PATCH and DELETE methods

In [ ]:
127.0.0.1:8000/api/hello_view/
    
#commit changes to github

# Section 9: Introduction to Viewsets

### 37. What is a Viewset?

Viewsets use model operations for functions  
Takes care of a lot of typical logic  
Perfect for standard database operations  
Fastest way to make a database interface  
Good for:  
Simple CRUD interfaces for db  
quick and simple API  
little to no customization on logic  
working with standard data structures  



### 38. Create a simple Viewset

In [ ]:
#add to views.py
from rest_framework import viewsets
...

class HelloViewSet(viewsets.ViewSet):
    """Test API ViewSet"""

    def list(self, request):
        """Return a hello message."""

        a_viewset = [
            'Uses actions (list, create, retrieve, update, partial_update)',
            'Automatically maps to URLS using Routers',
            'Provides more functionality with less code',
        ]

        return Response({'message': 'Hello!', 'a_viewset': a_viewset})

### 39. Add URL Router

In [ ]:
#register viewset with a default router into 
#add to views.py

...
from django.urls import path, include

from rest_framework.routers import DefaultRouter


router = DefaultRouter()
router.register('hello-viewset', views.HelloViewSet, base_name='hello-viewset')

urlpatterns = [
    path('hello-view/', views.HelloApiView.as_view()),
    path('', include(router.urls)),
]

### 40. Testing our Viewset

In [ ]:
http://127.0.0.1:8000/api/
#Router creates a page with all urls in router


### 41. Add create, retrieve, update, partial_update and destroy functions

In [ ]:
class HelloViewSet(viewsets.ViewSet):
    """Test API ViewSet"""
    serializer_class = serializers.HelloSerializer

    def list(self, request):
        """Return a hello message."""
        ...

    def create(self, request):
        """Create a new hello message."""
        serializer = self.serializer_class(data=request.data)

        if serializer.is_valid():
            name = serializer.validated_data.get('name')
            message = f'Hello {name}!'
            return Response({'message': message})
        else:
            return Response(
                serializer.errors,
                status=status.HTTP_400_BAD_REQUEST
            )

    def retrieve(self, request, pk=None):
        """Handle getting an object by its ID"""

        return Response({'http_method': 'GET'})

    def update(self, request, pk=None):
        """Handle updating an object"""

        return Response({'http_method': 'PUT'})

    def partial_update(self, request, pk=None):
        """Handle updating part of an object"""

        return Response({'http_method': 'PATCH'})

    def destroy(self, request, pk=None):
        """Handle removing an object"""

        return Response({'http_method': 'DELETE'})

 

### 42. Test Viewset

# Section 10: Create Profiles API

### 43. Plan our Profiles API

### 44. Create user profile serializer

### 45. Create profile ViewSet

### 46. Register profile Viewset with the URL router

### 47. Test creating a profile

### 48. Create permission class

### 49. Add authentication and permissions to ViewSet

### 50. Test new permissions

### 51. Add search profiles feature

### 52. Test searching profiles

# Section 11: Create login API

### 53. Create login API viewset

### 54. Test login API

### 55. Set token header using ModHeader extention

# Section 11: Create profile feed API

### 56. Plan profile feed API

### 57. Add new model Item

### 58. Create and run model migration

### 59. Add profile feed model to admin

### 60. Create profile feed item serializer

### 61. Create ViewSet for our profile feed item

### 62. Test Feed API

### 63. Add permissions for feed API

### 64. Test feed API permissions

### 65. Restrict viewing status updates to logged in users only

### 66. Test new private feed

# Section 13: Deploying our API to a server on AWS

### 67. Introduction to deploying our app to AWS

### 68. Add key pair to AWS

### 69. Create EC2 server instance

### 70. Add deployment script and configs to our project

### 71. Deploy to server

### 72. Update allowed hosts and deploy changes

# Section 14: Summary

### 73. Course outro